In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise import KNNBasic
from surprise.model_selection import train_test_split
from surprise import dump
import csv
from surprise import accuracy

In [44]:
# Path to dump file and name
dumpfile = os.path.join('../data/dump/dump_knn_pearsonbaseline_500dump_file')

beer_pickel_path = os.path.join('../data/dump/beer.pkl')

In [49]:
# Lets load our dump files
predictions,algo = dump.load(dumpfile)

beers_df = pd.read_pickle(beer_pickel_path)

In [46]:
def get_beer_name (beer_raw_id):
    beer_name = beers_df.loc[beers_df.beer_id==beer_raw_id,'name'].values[0]
    return beer_name

def get_beer_style (beer_raw_id):
    beer_style = beers_df.loc[beers_df.beer_id==beer_raw_id,'style'].values[0]
    return beer_style

def get_beer_score_mean (beer_raw_id):
    score_mean = beers_df.loc[beers_df.beer_id==beer_raw_id,'score'].values[0]
    return score_mean

def get_beer_neighbors (beer_raw_id):
    beer_inner_id = algo.trainset.to_inner_iid(beer_raw_id)
    beer_neighbors = algo.get_neighbors(beer_inner_id, k=10)
    beer_neighbors = (algo.trainset.to_raw_iid(inner_id)
                  for inner_id in beer_neighbors)
    return(beer_neighbors)

def get_beer_recc_df (beer_raw_id):
    beer_inner_id = algo.trainset.to_inner_iid(beer_raw_id)
    beer_neighbors = algo.get_neighbors(beer_inner_id, k=10)
    beer_neighbors = (algo.trainset.to_raw_iid(inner_id)
                      for inner_id in beer_neighbors)
    beers_id_recc = []
    beer_name_recc =[]
    beer_style_recc = []
    beer_score_mean = []
    for beer in beer_neighbors:
        beers_id_recc.append(beer)
        beer_name_recc.append(get_beer_name(beer))
        beer_style_recc.append(get_beer_style(beer))
        beer_score_mean.append(get_beer_score_mean(beer))
    beer_reccomendations_df = pd.DataFrame(list(zip(beers_id_recc,beer_name_recc,beer_style_recc,beer_score_mean)),
                                       columns=['beer_id', 'name', 'style', 'score_mean'])
    return beer_reccomendations_df

In [47]:
trainset = algo.trainset

In [48]:
# Lets test some beers.  Enter a beer and use the predictions model to return 5 nearest neighbors
# K=10

beer_raw_id =  232
print(f'The 10 nearest neighbors of {get_beer_name(beer_raw_id)}, {get_beer_style(beer_raw_id)},\
 score = {get_beer_score_mean (beer_raw_id)} are:')
df = get_beer_recc_df (beer_raw_id)
df.head(10)

The 10 nearest neighbors of Corona Extra, American Adjunct Lager, score = 2.2471090047393374 are:


,beer_id,name,style,score_mean
0,233,Corona Light,American Light Lager,1.884826
1,1320,Bud Light,American Light Lager,1.790027
2,3734,Michelob Ultra,American Light Lager,1.795152
3,65,Budweiser,American Adjunct Lager,2.261485
4,1321,Modelo Especial,American Adjunct Lager,2.763934
5,2803,Sol,American Adjunct Lager,2.391426
6,1907,Dos Equis Special Lager,American Adjunct Lager,2.682793
7,41821,Bud Light Lime,American Light Lager,2.078862
8,689,Red Stripe Jamaican Lager,American Adjunct Lager,2.926540
9,246,Heineken Lager Beer,European Pale Lager,2.641871
